![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [33]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

filename_features = "./Dados/UCI HAR Dataset/features.txt"
filename_labels   = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain   = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain   = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest  = "./Dados/UCI HAR Dataset/test/subject_test.txt"
filename_xtest    = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest    = "./Dados/UCI HAR Dataset/test/y_test.txt"

# --- Features ---
features = pd.read_csv(filename_features, sep=r"\s+", header=None, names=['id', 'nome_var'])
nomes_features = features['nome_var']

# Corrige duplicatas nos nomes
nomes_features = pd.Index(nomes_features)
if nomes_features.duplicated().any():
    nomes_features = nomes_features.to_series().where(~nomes_features.duplicated(),
                                                      nomes_features.to_series() + "_" + nomes_features.to_series().groupby(nomes_features).cumcount().astype(str))

nomes_features = nomes_features.tolist()

# --- Labels ---
labels = pd.read_csv(filename_labels, sep=r"\s+", header=None, names=['cod_label', 'label'])

# --- Conjunto de treino ---
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep=r"\s+", header=None, names=nomes_features)
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

# --- Conjunto de teste ---
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, sep=r"\s+", header=None, names=nomes_features)
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [3]:
print(f'X_train {X_train.shape}')
print(f'X_test {X_test.shape}')
print(f'y_train {y_train.shape}')
print(f'y_test {y_test.shape}')

X_train (7352, 561)
X_test (2947, 561)
y_train (7352, 1)
y_test (2947, 1)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [39]:
%%time

clf = DecisionTreeClassifier(random_state=42,
                             ccp_alpha=0.001).fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

print(f'Acurácia de treino:{train_score*100:.1f}')
print(f'Acurácia de teste:{test_score*100:.1f}\n')

Acurácia de treino:97.6
Acurácia de teste:88.0

CPU times: total: 4.02 s
Wall time: 4 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [35]:
%%time

prcomp = PCA(n_components=1).fit(X_train)

X_pca_train = prcomp.transform(X_train)
X_pca_test = prcomp.transform(X_test)

pca_clf = DecisionTreeClassifier(random_state=42,
                                 ccp_alpha=0.001).fit(X_pca_train, y_train)

train_score_pca = pca_clf.score(X_pca_train, y_train)
test_score_pca = pca_clf.score(X_pca_test, y_test)

print(f'Acurácia do train_score_pca é {train_score_pca*100:.1f}')
print(f'Acurácia do test_score_pca é {test_score_pca*100:.1f}\n')

Acurácia do train_score_pca é 50.0
Acurácia do test_score_pca é 45.7

CPU times: total: 703 ms
Wall time: 140 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [79]:
%%time

# Garante que o PCA seja rodado dentro do GridSearch
pipe = Pipeline([
    ('pca', PCA()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

grid_params = {
    'pca__n_components': [1,2,5,10,50],
    'clf__ccp_alpha': [0.0, 0.001, 0.01]}

grid = GridSearchCV(pipe, grid_params, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

print("Melhores parâmetros:", grid.best_params_)

# Avaliar no conjunto de teste
best_model = grid.best_estimator_
test_score_grid = best_model.score(X_test, y_test)
train_score_grid = best_model.score(X_train, y_train)
print(f'Acurácia no teste:{test_score_grid*100:.1f}')
print(f'Acurácia no treino: {train_score_grid*100:.1f}')

Melhores parâmetros: {'clf__ccp_alpha': 0.001, 'pca__n_components': 50}
Acurácia no teste:82.3
Acurácia no treino: 91.9
CPU times: total: 2.08 s
Wall time: 3.59 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

In [85]:
import time

resultados = []

# ----------------------
# SEM PCA
# ----------------------
start = time.time()

clf = DecisionTreeClassifier(random_state=42, ccp_alpha=0.001)
clf.fit(X_train, y_train)

train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)

end = time.time()

resultados.append({
    "Modelo": "Sem PCA",
    "Acurácia Treino": f'{train_score*100:.2f}%',
    "Acurácia Teste": f'{test_score*100:.2f}%',
    "Tempo Execução (s)": f'{end - start:.2f}s'
})


# ----------------------
# COM PCA (GridSearch)
# ----------------------
start = time.time()

pipe = Pipeline([
    ('pca', PCA()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

grid_params = {
    'pca__n_components': [1, 2, 5, 10, 50],
    'clf__ccp_alpha': [0.0, 0.001, 0.01]
}

grid = GridSearchCV(pipe, grid_params, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

train_score_grid = best_model.score(X_train, y_train)
test_score_grid = best_model.score(X_test, y_test)

end = time.time()

resultados.append({
    "Modelo": "Com PCA",
    "Acurácia Treino": f"{train_score_grid*100:.2f}%",
    "Acurácia Teste": f"{test_score_grid*100:.2f}%",
    "Tempo Execução (s)": f"{end - start:.2f}s"
})


# ----------------------
# Monta DataFrame
# ----------------------
df_resultados = pd.DataFrame(resultados)
df_resultados

,Modelo,Acurácia Treino,Acurácia Teste,Tempo Execução (s)
0,Sem PCA,97.58%,87.99%,3.95s
1,Com PCA,89.27%,82.42%,3.32s


## Interpretação

### SEM PCA
A acuracidade está maior, mas o tempo de execução é ligeiramente maior.

### Com PCA
A acuracidade caiu 8-5 pontos, mas o tempo foi menor

---
Isso exemplifica o dilema entre ```redução de dimensionalidade``` e ```perda de informação```